In [1]:
import os

os.environ["GRB_LICENSE_FILE"] = r"C:\Users\PC2\OneDrive\Desktop\gurobi.lic"

import gurobipy as gp
from gurobipy import GRB 


In [17]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from dataclasses import dataclass
from typing import Callable, List, Tuple, Optional, Dict, Any


# =========================================================
# Data structures
# =========================================================

@dataclass(frozen=True)
class StandardFormMILP:
    """
    Standard-form-ish MILP container.
    We support mixed senses (>=, <=, =) and mixed variable types.
    Objective: min c^T x
    Constraints: A x (sense) b
    """
    A: np.ndarray                 # shape (m,n)
    b: np.ndarray                 # shape (m,)
    sense: List[str]              # length m, each in {">=", "<=", "="}
    c: np.ndarray                 # shape (n,)
    lb: np.ndarray                # shape (n,)
    ub: np.ndarray                # shape (n,)
    vtype: List[str]              # length n, each in {"C","I","B"}


@dataclass(frozen=True)
class MutableSpec:
    """
    Which parameters are mutable, and their integer delta bounds.

    - b_idx: indices of RHS entries b_i that can change
    - c_idx: indices of objective coefficients c_j that can change
    - A_idx: list of (i,j) indices of A_ij coefficients that can change

    Delta bounds are uniform here for simplicity; you can extend to per-index bounds.
    """
    b_idx: List[int]
    c_idx: List[int]
    A_idx: List[Tuple[int,int]]
    delta_lb: int   # integer lower bound for each delta component
    delta_ub: int   # integer upper bound for each delta component


# =========================================================
# Model builder (standard form -> Gurobi)
# =========================================================

def build_forward_model(data: StandardFormMILP,
                        override: Optional[Dict[str, Any]] = None,
                        output_flag: int = 0):
    """
    Build a fresh Gurobi model from standard-form data.
    override can contain updated A, b, c (numpy arrays).
    Returns (model, x_vars, constrs, obj_expr).
    """
    A = data.A if override is None or "A" not in override else override["A"]
    b = data.b if override is None or "b" not in override else override["b"]
    c = data.c if override is None or "c" not in override else override["c"]

    m, n = A.shape
    model = gp.Model("forward")
    model.Params.OutputFlag = output_flag

    # Variables
    x = []
    for j in range(n):
        vt = data.vtype[j]
        if vt not in ("C", "I", "B"):
            raise ValueError(f"Invalid vtype[{j}]={vt}. Use 'C','I','B'.")
        xj = model.addVar(lb=float(data.lb[j]), ub=float(data.ub[j]),
                          vtype={"C":GRB.CONTINUOUS,"I":GRB.INTEGER,"B":GRB.BINARY}[vt],
                          name=f"x[{j}]")
        x.append(xj)

    # Objective
    obj = gp.quicksum(float(c[j]) * x[j] for j in range(n))
    model.setObjective(obj, GRB.MINIMIZE)

    # Constraints
    constrs = []
    for i in range(m):
        lhs = gp.quicksum(float(A[i,j]) * x[j] for j in range(n))
        s = data.sense[i]
        if s == ">=":
            ci = model.addConstr(lhs >= float(b[i]), name=f"con[{i}]")
        elif s == "<=":
            ci = model.addConstr(lhs <= float(b[i]), name=f"con[{i}]")
        elif s == "=":
            ci = model.addConstr(lhs == float(b[i]), name=f"con[{i}]")
        else:
            raise ValueError(f"Invalid sense[{i}]={s}. Use '>=','<=','='.")
        constrs.append(ci)

    model.optimize()
    return model, x, constrs, obj


# =========================================================
# Apply deltas to (A,b,c)
# =========================================================

def apply_deltas(base: StandardFormMILP,
                 spec: MutableSpec,
                 delta_b: np.ndarray,
                 delta_c: np.ndarray,
                 delta_A: np.ndarray):
    """
    Produces new (A,b,c) = (A0+ΔA, b0+Δb, c0+Δc) based on spec ordering.
    """
    A = base.A.copy()
    b = base.b.copy()
    c = base.c.copy()

    for k, i in enumerate(spec.b_idx):
        b[i] = b[i] + delta_b[k]
    for k, j in enumerate(spec.c_idx):
        c[j] = c[j] + delta_c[k]
    for k, (i,j) in enumerate(spec.A_idx):
        A[i,j] = A[i,j] + delta_A[k]

    return A, b, c


# =========================================================
# Witness check for WCE (existential / optimistic)
# =========================================================

def witness_exists(base: StandardFormMILP,
                   A: np.ndarray, b: np.ndarray, c: np.ndarray,
                   theta: float,
                   desired_space_cb: Callable[[gp.Model, List[gp.Var], List[gp.Constr]], None],
                   tol: float = 1e-6,
                   output_flag: int = 0):
    """
    Returns (True, witness_solution_dict) if exists x in desired space that is LL-optimal:
      feasible for (A,b), satisfies desired constraints, and objective <= theta+tol.
    """
    model, x, constrs, obj = build_forward_model(
        StandardFormMILP(A=A, b=b, sense=base.sense, c=c, lb=base.lb, ub=base.ub, vtype=base.vtype),
        override=None,
        output_flag=output_flag
    )

    # Add desired space constraints
    desired_space_cb(model, x, constrs)

    # Optimality cap
    model.addConstr(obj <= theta + tol, name="optimality_cap")

    model.optimize()
    if model.Status == GRB.OPTIMAL:
        sol = {"obj": model.ObjVal, "x": np.array([v.X for v in x])}
        return True, sol
    return False, None


# =========================================================
# Master over delta variables with no-good cuts (enumerative CCG baseline)
# =========================================================

def build_master(spec: MutableSpec,
                 output_flag: int = 0):
    """
    Master decision variables are integer deltas for all mutable parameters.
    Objective: L1 norm of all deltas.
    Returns (model, delta_vars, abs_vars_plus_minus) for later no-good cuts.
    """
    model = gp.Model("master")
    model.Params.OutputFlag = output_flag

    # Delta variables
    db = [model.addVar(vtype=GRB.INTEGER, lb=spec.delta_lb, ub=spec.delta_ub, name=f"db[{k}]")
          for k in range(len(spec.b_idx))]
    dc = [model.addVar(vtype=GRB.INTEGER, lb=spec.delta_lb, ub=spec.delta_ub, name=f"dc[{k}]")
          for k in range(len(spec.c_idx))]
    dA = [model.addVar(vtype=GRB.INTEGER, lb=spec.delta_lb, ub=spec.delta_ub, name=f"dA[{k}]")
          for k in range(len(spec.A_idx))]

    # L1 linearization: sum |delta|
    def add_abs(v: gp.Var, name: str):
        p = model.addVar(lb=0.0, name=f"{name}_p")
        m = model.addVar(lb=0.0, name=f"{name}_m")
        model.addConstr(v == p - m, name=f"{name}_abs")
        return p + m

    abs_terms = []
    for k, v in enumerate(db):
        abs_terms.append(add_abs(v, f"abs_db[{k}]"))
    for k, v in enumerate(dc):
        abs_terms.append(add_abs(v, f"abs_dc[{k}]"))
    for k, v in enumerate(dA):
        abs_terms.append(add_abs(v, f"abs_dA[{k}]"))

    model.setObjective(gp.quicksum(abs_terms), GRB.MINIMIZE)
    model.update()
    return model, db, dc, dA


def add_no_good_cut(model: gp.Model,
                    vars_list: List[gp.Var],
                    values: List[int],
                    name: str):
    """
    Correct no-good cut for bounded integer variables:
    exclude exactly (vars_list == valuesvalues.

    Enforces: exists k such that v_k <= val_k-1 OR v_k >= val_k+1.
    Implemented via two indicator binaries per component.
    """
    assert len(vars_list) == len(values)

    if len(vars_list) == 0:
        model.addConstr(0 >= 1, name=name)
        return

    diffs = []
    for k, (v, val) in enumerate(zip(vars_list, values)):
        u = model.addVar(vtype=GRB.BINARY, name=f"{name}_u[{k}]")  # v >= val+1
        l = model.addVar(vtype=GRB.BINARY, name=f"{name}_l[{k}]")  # v <= val-1
        model.addConstr(u + l <= 1, name=f"{name}_one_side[{k}]")

        # If u=1 => v >= val+1
        model.addGenConstrIndicator(u, True, v >= val + 1, name=f"{name}_ge[{k}]")
        # If l=1 => v <= val-1
        model.addGenConstrIndicator(l, True, v <= val - 1, name=f"{name}_le[{k}]")

        diffs.append(u + l)

    model.addConstr(gp.quicksum(diffs) >= 1, name=f"{name}_exclude_point")
    model.update()



# =========================================================
# General WCE solver (enumerative CCG baseline)
# =========================================================

def solve_wce_general(base: StandardFormMILP,
                      spec: MutableSpec,
                      desired_space_cb: Callable[[gp.Model, List[gp.Var], List[gp.Constr]], None],
                      max_iter: int = 50,
                      tol: float = 0.0,
                      master_output: int = 0,
                      oracle_output: int = 0):
    """
    General WCE solver:
      min ||ΔΩ||_1 s.t. exists optimal LL solution in desired space.

    Returns dict with best solution found, or None if none found within bounds/iterations.
    """
    master, db_vars, dc_vars, dA_vars = build_master(spec, output_flag=master_output)

    UB = float("inf")
    best = None

    for it in range(max_iter):
        master.optimize()
        if master.Status != GRB.OPTIMAL:
            break

        # Read deltas
        db_val = np.array([int(round(v.X)) for v in db_vars], dtype=int)
        dc_val = np.array([int(round(v.X)) for v in dc_vars], dtype=int)
        dA_val = np.array([int(round(v.X)) for v in dA_vars], dtype=int)
        dist = master.ObjVal

        # Apply deltas
        A_new, b_new, c_new = apply_deltas(base, spec, db_val, dc_val, dA_val)

        # Oracle: solve forward to get theta
        fwd_model, x_vars, constrs, obj_expr = build_forward_model(
            StandardFormMILP(A=A_new, b=b_new, sense=base.sense, c=c_new, lb=base.lb, ub=base.ub, vtype=base.vtype),
            override=None,
            output_flag=oracle_output
        )
        if fwd_model.Status != GRB.OPTIMAL:
            # treat as failure; cut this delta vector
            all_master_vars = db_vars + dc_vars + dA_vars
            all_vals = list(db_val) + list(dc_val) + list(dA_val)
            add_no_good_cut(master, all_master_vars, all_vals, name=f"nogood_infeas[{it}]")
            continue

        theta = fwd_model.ObjVal

        # Witness check
        ok, witness = witness_exists(base, A_new, b_new, c_new, theta, desired_space_cb,
                                     tol=1e-6, output_flag=oracle_output)

        if ok:
            if dist < UB:
                UB = dist
                best = {
                    "dist": UB,
                    "db": db_val, "dc": dc_val, "dA": dA_val,
                    "A": A_new, "b": b_new, "c": c_new,
                    "theta": theta,
                    "witness": witness,
                }
            # Since master is minimizing dist, if we found a witness at current optimum dist,
            # we are done (LB==UB) for this enumerative master.
            if UB - dist <= tol:
                break

        # If witness fails, exclude this delta vector
        all_master_vars = db_vars + dc_vars + dA_vars
        all_vals = list(db_val) + list(dc_val) + list(dA_val)
        add_no_good_cut(master, all_master_vars, all_vals, name=f"nogood[{it}]")

    return best


# =========================================================
# Demo on your toy (now in standard form)
# =========================================================

def demo():
    # Toy variables: x1,x2 continuous, y1,y2 binary
    # Put them into x vector as [x1,x2,y1,y2]
    # Objective: 2x1 + 1x2 + 3y1 + 1y2
    c0 = np.array([2, 1, 3, 1], dtype=float)

    # Constraints:
    # (1) x1 + x2 >= b
    # (2) x1 - 5 y1 <= 0
    # (3) x2 - 5 y2 <= 0
    # Standard form with mixed senses:
    A0 = np.array([
        [1, 1, 0, 0],
        [1, 0, -5, 0],
        [0, 1, 0, -5],
    ], dtype=float)
    sense = [">=", "<=", "<="]

    b0 = np.array([4, 0, 0], dtype=float)

    lb = np.array([0, 0, 0, 0], dtype=float)
    ub = np.array([GRB.INFINITY, GRB.INFINITY, 1, 1], dtype=float)
    vtype = ["C", "C", "B", "B"]

    base = StandardFormMILP(A=A0, b=b0, sense=sense, c=c0, lb=lb, ub=ub, vtype=vtype)

    # Desired space: y1 == 1  (x[2] is y1)
    def desired_cb(model: gp.Model, xvars: List[gp.Var], constrs: List[gp.Constr]):
        model.addConstr(xvars[2] == 1, name="desired_y1_on")
        model.addConstr(xvars[0] <= 1.5, name="desired_x2_limit")

    # Mutable spec: allow changing ONLY the RHS of constraint (1) (b[0]) by integer deltas
    spec = MutableSpec(
        b_idx=[0,1],     # only demand RHS mutable
        c_idx=[],      # no cost changes
        A_idx=[],      # no A changes
        delta_lb=-10,
        delta_ub=+10
    )

    best = solve_wce_general(
        base=base,
        spec=spec,
        desired_space_cb=desired_cb,
        max_iter=50,
        master_output=0,
        oracle_output=0
    )

    if best is None:
        print("No WCE found.")
        return

    print("=== BEST WCE FOUND ===")
    print("L1 dist:", best["dist"])
    print("delta b:", best["db"])
    print("new b:", best["b"])
    print("new A:", best["A"])
    print("new c:", best["c"])
    print("theta:", best["theta"])
    print("witness obj:", best["witness"]["obj"])
    print("witness x:", best["witness"]["x"])


if __name__ == "__main__":
    demo()


=== BEST WCE FOUND ===
L1 dist: 1.0
delta b: [ 0 -1]
new b: [ 4. -1.  0.]
new A: [[ 1.  1.  0.  0.]
 [ 1.  0. -5.  0.]
 [ 0.  1.  0. -5.]]
new c: [2. 1. 3. 1.]
theta: 8.0
witness obj: 8.0
witness x: [0. 4. 1. 1.]


In [18]:
def solve_forward_only(base: StandardFormMILP, A: np.ndarray, b: np.ndarray, c: np.ndarray, output_flag: int = 0):
    """
    Solve the forward MILP once and return theta and primal x.
    """
    fwd_model, x_vars, constrs, obj_expr = build_forward_model(
        StandardFormMILP(A=A, b=b, sense=base.sense, c=c, lb=base.lb, ub=base.ub, vtype=base.vtype),
        override=None,
        output_flag=output_flag
    )
    if fwd_model.Status != GRB.OPTIMAL:
        return None, None
    x = np.array([v.X for v in x_vars])
    return fwd_model.ObjVal, x


def compute_l1_dist_from_deltas(db: np.ndarray, dc: np.ndarray, dA: np.ndarray):
    return float(np.sum(np.abs(db)) + np.sum(np.abs(dc)) + np.sum(np.abs(dA)))


def sanity_check_wce_result(base: StandardFormMILP,
                            spec: MutableSpec,
                            desired_space_cb: Callable[[gp.Model, List[gp.Var], List[gp.Constr]], None],
                            best: Dict[str, Any],
                            tol: float = 1e-6,
                            output_flag: int = 0):
    """
    Sanity-check that 'best' is a valid WCE for the forward model:
      1) best['dist'] matches |deltas|_1
      2) theta is the forward optimum under (A,b,c)
      3) witness exists in desired space with obj <= theta + tol
      4) witness objective approximately equals theta (optional check)
    """
    assert best is not None, "best is None (no WCE found)."

    # 1) Check distance
    dist_calc = compute_l1_dist_from_deltas(best["db"], best["dc"], best["dA"])
    print("=== SANITY 1: Distance ===")
    print(f"reported dist = {best['dist']}, computed dist = {dist_calc}")
    if abs(best["dist"] - dist_calc) > 1e-6:
        print("WARNING: dist mismatch. (If master used weighted distance, this is expected.)")

    # 2) Re-solve forward
    print("\n=== SANITY 2: Forward optimality ===")
    theta_star, x_star = solve_forward_only(base, best["A"], best["b"], best["c"], output_flag=output_flag)
    if theta_star is None:
        raise RuntimeError("Forward model at best parameters is not optimal/feasible.")
    print(f"theta(recomputed) = {theta_star:.6f} | theta(stored) = {best['theta']:.6f} | gap = {best['theta'] - theta_star:.3e}")

    # 3) Witness existence
    print("\n=== SANITY 3: Witness exists in desired space at optimum ===")
    ok, wit = witness_exists(
        base=base,
        A=best["A"],
        b=best["b"],
        c=best["c"],
        theta=theta_star,
        desired_space_cb=desired_space_cb,
        tol=tol,
        output_flag=output_flag
    )
    print("witness_exists ?", ok)
    if not ok:
        raise RuntimeError("Sanity failed: no witness exists, so this is NOT a WCE.")

    print(f"witness obj = {wit['obj']:.6f} | theta = {theta_star:.6f} | wit-theta = {wit['obj'] - theta_star:.3e}")

    # 4) Optional: check witness objective approximately equals theta (it should, given your formulation)
    if wit["obj"] > theta_star + 1e-5:
        print("WARNING: witness objective is above theta; check your witness constraint/tolerance.")

    print("\n=== SANITY PASSED: This is a valid Weak Optimality CE ===")
    return {"theta": theta_star, "x_opt": x_star, "witness": wit}


In [23]:
def demo():
    # -------------------------------
    # 1) Define the base MILP
    # -------------------------------
    c0 = np.array([2, 1, 3, 1], dtype=float)

    A0 = np.array([
        [1, 1, 0, 0],
        [1, 0, -5, 0],
        [0, 1, 0, -5],
    ], dtype=float)

    sense = [">=", "<=", "<="]
    b0 = np.array([4, 0, 0], dtype=float)

    lb = np.array([0, 0, 0, 0], dtype=float)
    ub = np.array([GRB.INFINITY, GRB.INFINITY, 1, 1], dtype=float)
    vtype = ["C", "C", "B", "B"]

    base = StandardFormMILP(
        A=A0, b=b0, sense=sense,
        c=c0, lb=lb, ub=ub, vtype=vtype
    )

    # -------------------------------
    # 2) Desired space
    # -------------------------------
    def desired_cb(model, xvars, constrs):
        # y1 == 1  (x[2])
        model.addConstr(xvars[2] == 1, name="desired_y1_on")

    # -------------------------------
    # 3) Mutable specification
    # -------------------------------
    spec = MutableSpec(
        b_idx=[0],     # only demand RHS
        c_idx=[],
        A_idx=[],
        delta_lb=-10,
        delta_ub=10
    )

    # -------------------------------
    # 4) Solve WCE
    # -------------------------------
    best = solve_wce_general(
        base=base,
        spec=spec,
        desired_space_cb=desired_cb,
        max_iter=50,
        master_output=0,
        oracle_output=0
    )

    if best is None:
        print("No WCE found.")
        return

    print("\n=== BEST WCE FOUND ===")
    print("L1 dist:", best["dist"])
    print("delta b:", best["db"])
    print("new b:", best["b"])
    print("theta:", best["theta"])
    print("witness obj:", best["witness"]["obj"])
    print("witness x:", best["witness"]["x"])

    # -------------------------------
    # 5) Sanity check
    # -------------------------------
    print("\n=== RUN SANITY CHECK ===")
    sanity_check_wce_result(
        base=base,
        spec=spec,
        desired_space_cb=desired_cb,
        best=best,
        tol=1e-6,
        output_flag=0
    )


# ✅ ONLY THIS at top level
demo()



=== BEST WCE FOUND ===
L1 dist: 2.0
delta b: [2]
new b: [6. 0. 0.]
theta: 11.0
witness obj: 11.0
witness x: [1. 5. 1. 1.]

=== RUN SANITY CHECK ===
=== SANITY 1: Distance ===
reported dist = 2.0, computed dist = 2.0

=== SANITY 2: Forward optimality ===
theta(recomputed) = 11.000000 | theta(stored) = 11.000000 | gap = 0.000e+00

=== SANITY 3: Witness exists in desired space at optimum ===
witness_exists ? True
witness obj = 11.000000 | theta = 11.000000 | wit-theta = 0.000e+00

=== SANITY PASSED: This is a valid Weak Optimality CE ===
